In [1]:
import pandas as pd
import numpy as np

import spacy
import nltk

from collections import Counter

In [3]:
# пришлось провести манипуляции из 1 урока отдельно с train и test, чтоб иметь 2 раздельных датасета
# также я выбросил шаг с lower, т.к. nltk чувствителен к регистру
train_df = pd.read_pickle("/content/train_result.pkl")
test_df = pd.read_pickle("/content/test_result.pkl")

In [ ]:
train_df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...,"[when, a, father, is, dysfunctional, and, is, ...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i cannot use cause they...,"[thanks, for, lyft, credit, i, can, not, use, ...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0,#model i love u take with u all the time in ...,model i love yoyou take with yoyou all the tim...,"[model, i, love, yoyou, take, with, yoyou, all...","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]"
4,5,0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


In [ ]:
test_df.head()

,id,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,31963,#studiolife #aislife #requires #passion #dedic...,studiolife aislife requires passion dedication...,"[studiolife, aislife, requires, passion, dedic...","[studiolife, aislife, requires, passion, dedic...","[studiolif, aislif, requir, passion, dedic, wi...","[studiolife, aislife, requires, passion, dedic..."
1,31964,@user #white #supremacists want everyone to s...,white supremacists want everyone to see the ne...,"[white, supremacists, want, everyone, to, see,...","[white, supremacists, want, everyone, see, new...","[white, supremacist, want, everyon, see, new, ...","[white, supremacist, want, everyone, see, new,..."
2,31965,safe ways to heal your #acne!! #altwaystohe...,safe ways to heal your acne altwaystoheal heal...,"[safe, ways, to, heal, your, acne, altwaystohe...","[safe, ways, heal, acne, altwaystoheal, health...","[safe, way, heal, acn, altwaystoh, healthi, heal]","[safe, way, heal, acne, altwaystoheal, healthy..."
3,31966,is the hp and the cursed child book up for res...,is the hp and the cursed child book up for res...,"[is, the, hp, and, the, cursed, child, book, u...","[hp, cursed, child, book, reservations, alread...","[hp, curs, child, book, reserv, alreadi, ye, h...","[hp, cursed, child, book, reservation, already..."
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",rd bihday to my amazing hilarious nephew eli a...,"[rd, bihday, to, my, amazing, hilarious, nephe...","[rd, bihday, amazing, hilarious, nephew, eli, ...","[rd, bihday, amaz, hilari, nephew, eli, ahmir,...","[rd, bihday, amazing, hilarious, nephew, eli, ..."


### Задача 1

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
def count_ners(data):
  cnt = Counter()
  ners = {}

  for ct in data:
    doc = nlp(ct)
    for e in doc.ents:
      cnt[e.text] += 1
      ners[e.text] = e.label_

  result = [(k, v, cnt[k]) for (k,v) in ners.items()]

  return pd.DataFrame(result, columns=['Word', 'NER', 'Count'])

In [ ]:
train_ners = count_ners(train_df['clean_tweet'])

train_ners.head()

,Word,NER,Count
0,pdx,ORG,1
1,bihday,DATE,694
2,tomorrow,DATE,387
3,the next school year,DATE,1
4,the year,DATE,17


In [ ]:
train_ners.sort_values('Count', ascending=False).head(20)

,Word,NER,Count
29,today,DATE,1016
1,bihday,DATE,694
14,friday,DATE,439
2,tomorrow,DATE,387
42,first,ORDINAL,338
113,summer,DATE,332
20,sunday,DATE,331
73,one,CARDINAL,325
26,tonight,TIME,292
21,s day,DATE,291


Видим, что в train чаще всего нам попадались даты, числительные и географические объекты.

In [ ]:
train_ners[train_ners['NER'].isin(['GPE', 'ORG', 'PERSON'])].sort_values('Count', ascending=False).head(20)

,Word,NER,Count
132,america,GPE,157
374,orlando,GPE,131
283,london,GPE,117
444,tbt,ORG,100
48,selfie,GPE,89
591,us,GPE,87
111,aww,ORG,81
54,florida,GPE,68
269,uk,GPE,65
361,india,GPE,53


Если ограничить выборку по классу, то имеем приоритет со стороны географических объектов, но так же множество организаций.
  

С определением организаций есть определённая сложность - многие междометия попали в список организаций (aww, tbt).

Персоны, в основном, определены верно.

Посмотрим на тест

In [ ]:
test_ners = count_ners(test_df['clean_tweet'])

test_ners.head()

,Word,NER,Count
0,rd bihday,ORG,1
1,ahmir uncle dave,PERSON,1
2,neuro bass lifestyle,PERSON,1
3,one,CARDINAL,162
4,half,CARDINAL,22


In [ ]:
test_ners.sort_values('Count', ascending=False).head(20)

,Word,NER,Count
6,today,DATE,564
26,bihday,DATE,376
25,friday,DATE,206
28,tomorrow,DATE,198
57,first,ORDINAL,195
175,s day,DATE,170
235,tonight,TIME,166
17,summer,DATE,164
3,one,CARDINAL,162
44,sunday,DATE,156


В test чаще всего нам попадались даты, числительные и географические объекты.

In [ ]:
test_ners[test_ners['NER'].isin(['GPE', 'ORG', 'PERSON'])].sort_values('Count', ascending=False).head(20)

,Word,NER,Count
290,orlando,GPE,81
148,tbt,ORG,70
8,london,GPE,65
55,us,GPE,60
119,america,GPE,58
128,aww,ORG,39
110,trump,PERSON,36
315,selfie,PERSON,33
918,japan,GPE,30
663,florida,GPE,29


Ситуация идентична train

## Задание 2

In [4]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [12]:
def count_ners_nltk(data):
  cnt = Counter()
  ners = {}

  for ct in data:
    for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(ct))):
      if hasattr(chunk, 'label'):
        cnt[' '.join(c[0] for c in chunk)] += 1
        ners[' '.join(c[0] for c in chunk)] = chunk.label()

  result = [(k, v, cnt[k]) for (k,v) in ners.items()]

  return pd.DataFrame(result, columns=['Word', 'NER', 'Count'])

In [13]:
nltk_nres_train = count_ners_nltk(train_df['clean_tweet'])

In [14]:
nltk_nres_train.head()

,Word,NER,Count


Почему-то пусто

In [16]:
nltk_nres_test = count_ners_nltk(test_df['clean_tweet'])

In [17]:
nltk_nres_test.head()

,Word,NER,Count


И тут тоже. Почему?

Я проверял nltk на фразах, где есть uppercase (например, имена или названия начинаются с большой буквы) - и там всё работает

In [20]:
count_ners_nltk(['hello Mr Anderson how do you like New York'])

,Word,NER,Count
0,Anderson,PERSON,1
1,New York,GPE,1


Вот пример. Видимо, в твитах с такими данными проблема.

На мой взгляд, Spacy работает лучше.  

С другой стороны, он довольно много ошибается на странных и незнакомых ему словах